## Setup File

Import function

In [ ]:
import os
import shutil
import zipfile
import pickle
import re
import csv
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Create Folders if not existing

In [ ]:
#File names for the angel and power data
a_data = 'angle_data.txt' #The angel data 
p_data = 'power_data.csv' #The power data

#Path to the different folders
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'

if not os.path.isdir(data_path):
    os.mkdir(data_path)
    os.mkdir(temp_path)
    os.mkdir(backup_path)

## Preprocessing

Preprocess Robotic arms trejectory

In [ ]:
with open(a_data, 'r') as file:
    try:
        data = file.read()
        file.close()
    except UnicodeDecodeError:
        file.close()
data = data.splitlines()
data = data[4:len(data)-1]

traj_data = []
for file in data:
    traj_data = traj_data + [[float(n) for n in file.split()]]

Preprocess Voltage/current Measurements

In [ ]:
f = open(p_data)

data = []
for line in f:
    data_line = line.rstrip().split('\t')
    data.append(data_line)

Volt_Amp_data = []
for file in data[9:]:
    file = file[0].replace(',','.')
    file = file.replace(' ','')
    Volt_Amp_data = Volt_Amp_data + [[float(n) for n in file.split(';')[:len(file.split(';'))-1]]]

Preprocessing "Calculate Power form Voltage/current measurements

In [ ]:
power_data = []
for sample in Volt_Amp_data:
    power = np.abs(sample[1]*sample[4]) \
          + np.abs(sample[2]*sample[5]) \
          + np.abs(sample[3]*sample[6])
    power_data.append([sample[0], power])

Save the data

In [ ]:
with open(data_path + '/joint_angle_data.pickle', 'wb') as file:
    pickle.dump(traj_data, file)
    file.close()
with open(data_path + '/power_data.pickle', 'wb') as file:
    pickle.dump(power_data, file)
    file.close()

Figure ploting (testing)

In [ ]:
plt.figure(figsize=(12,6))
fig = plt.subplot()
x = range(len(power_data))
fig.plot(x, power_data,  label='power',  marker='o',linewidth=0.3, markersize=1.5)
fig.legend();
fig.set_ylabel('Power [W]')
fig.set_xlabel('Time [0.01s]')
fig.set_title('Total Power');